In [1]:
# O PySpark suporta dois formatos principais de arquivos JSON:

import json


# 1. JSON de linha única (single-line JSON), onde todo o conteúdo está dentro de uma lista ([ ... ]). Esse formato é geralmente usado para pequenos arquivos JSON.
example_data = [
    {"id": 1, "nome": "Valter", "idade": 28, "cidade": "Sao Paulo"},
    {"id": 2, "nome": "Ana", "idade": 34, "cidade": "Rio de Janeiro"},
    {"id": 3, "nome": "Carlos", "idade": 25, "cidade": "Belo Horizonte"},
    {"id": 4, "nome": "Fernanda", "idade": 40, "cidade": "Porto Alegre"},
    {"id": 5, "nome": "Roberto", "idade": 31, "cidade": "Curitiba"}
]
# Salvar um JSON de teste
with open("clientes_single_line.json", "w") as file:
    json.dump(example_data, file)


# 2. JSON de linha (line-delimited JSON), onde cada objeto JSON está em uma linha separada. 
# Esse formato é mais eficiente para grandes conjuntos de dados e é o padrão usado pelo PySpark para leitura de JSON.
# Dados
data = [
    {"id": 1, "nome": "Valter", "idade": 28, "cidade": "Sao Paulo"},
    {"id": 2, "nome": "Ana", "idade": 34, "cidade": "Rio de Janeiro"},
    {"id": 3, "nome": "Carlos", "idade": 25, "cidade": "Belo Horizonte"},
    {"id": 4, "nome": "Fernanda", "idade": 40, "cidade": "Porto Alegre"},
    {"id": 5, "nome": "Roberto", "idade": 31, "cidade": "Curitiba"}
]
# Salvar em JSON de linha
with open("clientes_line_delimited.json", "w") as file:
    for record in data:
        file.write(json.dumps(record) + "\n")


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, sum

In [3]:
# Iniciar uma sessão Spark
spark = SparkSession.builder.appName("Demo").getOrCreate()

24/11/24 09:23:28 WARN Utils: Your hostname, Mac-mini-de-Valter.local resolves to a loopback address: 127.0.0.1; using 192.168.0.100 instead (on interface en0)
24/11/24 09:23:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/24 09:23:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Leitura do arquivo CSV
df_vendas = spark.read.csv("vendas.csv", header=True, inferSchema=True)
df_vendas.show()

+---+---------------+----------+------+
| id|        produto|quantidade| valor|
+---+---------------+----------+------+
|  1|       Notebook|         2|2500.0|
|  2|     Smartphone|         5|1000.0|
|  3|         Tablet|         3| 800.0|
|  4|Fones de ouvido|        10| 150.0|
|  5|     Smartwatch|         4| 500.0|
+---+---------------+----------+------+



In [5]:
# Leitura do arquivo JSON
df_single_line = spark.read.json("clientes_single_line.json")
df_single_line.show()

+--------------+---+-----+--------+
|        cidade| id|idade|    nome|
+--------------+---+-----+--------+
|     Sao Paulo|  1|   28|  Valter|
|Rio de Janeiro|  2|   34|     Ana|
|Belo Horizonte|  3|   25|  Carlos|
|  Porto Alegre|  4|   40|Fernanda|
|      Curitiba|  5|   31| Roberto|
+--------------+---+-----+--------+



In [6]:
# Tente ler o novo JSON
df_line_delimited = spark.read.json("clientes_line_delimited.json")
df_line_delimited.show()

+--------------+---+-----+--------+
|        cidade| id|idade|    nome|
+--------------+---+-----+--------+
|     Sao Paulo|  1|   28|  Valter|
|Rio de Janeiro|  2|   34|     Ana|
|Belo Horizonte|  3|   25|  Carlos|
|  Porto Alegre|  4|   40|Fernanda|
|      Curitiba|  5|   31| Roberto|
+--------------+---+-----+--------+



In [7]:
# Exibir os esquemas dos DataFrames
print("Esquema do DataFrame de Vendas:")
df_vendas.printSchema()

print("Esquema do DataFrame de Clientes:")
df_line_delimited.printSchema()

# Exibição dos dados:
print("Dados do DataFrame de Vendas:")
df_vendas.show()

print("Dados do DataFrame de Clientes:")
df_line_delimited.show()

Esquema do DataFrame de Vendas:
root
 |-- id: integer (nullable = true)
 |-- produto: string (nullable = true)
 |-- quantidade: integer (nullable = true)
 |-- valor: double (nullable = true)

Esquema do DataFrame de Clientes:
root
 |-- cidade: string (nullable = true)
 |-- id: long (nullable = true)
 |-- idade: long (nullable = true)
 |-- nome: string (nullable = true)

Dados do DataFrame de Vendas:
+---+---------------+----------+------+
| id|        produto|quantidade| valor|
+---+---------------+----------+------+
|  1|       Notebook|         2|2500.0|
|  2|     Smartphone|         5|1000.0|
|  3|         Tablet|         3| 800.0|
|  4|Fones de ouvido|        10| 150.0|
|  5|     Smartwatch|         4| 500.0|
+---+---------------+----------+------+

Dados do DataFrame de Clientes:
+--------------+---+-----+--------+
|        cidade| id|idade|    nome|
+--------------+---+-----+--------+
|     Sao Paulo|  1|   28|  Valter|
|Rio de Janeiro|  2|   34|     Ana|
|Belo Horizonte|  3|   2

In [8]:
#5. Filtragem:
# Filtrar vendas com quantidade maior que 3
df_vendas_filtrado = df_vendas.filter(col("quantidade") > 3)
print("Vendas com quantidade maior que 3:")
df_vendas_filtrado.show()

# Filtrar clientes de Sao Paulo
df_clientes_filtrado = df_line_delimited.filter(col("cidade") == "Sao Paulo")
print("Clientes de Sao Paulo:")
df_clientes_filtrado.show()

#6. Agregação:
# Calcular o valor total de vendas
valor_total = df_vendas.agg(sum(df_vendas["valor"] * df_vendas["quantidade"])).alias("valor_total")
print("Valor total de vendas:")
valor_total.show()

# Calcular a idade média dos clientes
idade_media = df_line_delimited.agg(avg(df_line_delimited["idade"])).alias("idade_media")
print("Idade média dos clientes:")
idade_media.show()

# União:
# Criar um novo DataFrame com informações adicionais de vendas
df_vendas_adicionais = spark.createDataFrame([
    (6, "Monitor", 2, 800.00),
    (7, "Teclado", 5, 100.00),
    (8, "Mouse", 3, 50.00),
], ["id", "produto", "quantidade", "valor"])

# Unir os DataFrames de vendas
df_vendas_uniao = df_vendas.union(df_vendas_adicionais)
print("Vendas unidas:")
print(df_vendas_uniao.show())

# Escrita de dados:

# Escrever o DataFrame unificado em um arquivo CSV
df_vendas_uniao.write.csv("exportacao_vendas.csv", header=True, mode="overwrite")

# Escrever o DataFrame de clientes filtrado em um arquivo JSON
df_clientes_filtrado.write.json("exportacao_clientes.json", mode="overwrite")

print("Arquivo Exportados com sucesso!")

# Parar a sessão Spark
spark.stop()

Vendas com quantidade maior que 3:
+---+---------------+----------+------+
| id|        produto|quantidade| valor|
+---+---------------+----------+------+
|  2|     Smartphone|         5|1000.0|
|  4|Fones de ouvido|        10| 150.0|
|  5|     Smartwatch|         4| 500.0|
+---+---------------+----------+------+

Clientes de Sao Paulo:
+---------+---+-----+------+
|   cidade| id|idade|  nome|
+---------+---+-----+------+
|Sao Paulo|  1|   28|Valter|
+---------+---+-----+------+

Valor total de vendas:
+-------------------------+
|sum((valor * quantidade))|
+-------------------------+
|                  15900.0|
+-------------------------+

Idade média dos clientes:
+----------+
|avg(idade)|
+----------+
|      31.6|
+----------+

Vendas unidas:
+---+---------------+----------+------+
| id|        produto|quantidade| valor|
+---+---------------+----------+------+
|  1|       Notebook|         2|2500.0|
|  2|     Smartphone|         5|1000.0|
|  3|         Tablet|         3| 800.0|
|  4